In [100]:
import pandas as pd

file_path='train.csv'
data=pd.read_csv(file_path)
print(data)
print(data.describe())
print(data.shape)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
5              6         0       3   
6              7         0       1   
7              8         0       3   
8              9         1       3   
9             10         1       2   
10            11         1       3   
11            12         1       1   
12            13         0       3   
13            14         0       3   
14            15         0       3   
15            16         1       2   
16            17         0       3   
17            18         1       2   
18            19         0       3   
19            20         1       3   
20            21         0       2   
21            22         1       2   
22            23         1       3   
23            24         1       1   
24            25         0       3   
25          

In [3]:
#find number of missing values in each feature
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

# Data Preprocessing

Remove columns not required for data modeling

In [81]:
train_data=data.drop(columns=['PassengerId','Name'])
train_data.head()
print(train_data.dtypes)

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object


find if data is imbalanced

In [101]:
#find instances belonging to class Survived=0 and Survived=1
n=train_data.shape[0]
print(n)
no=train_data.Survived.value_counts()[0]
yes=train_data.Survived.value_counts()[1]
print((yes/n)*100)
print((no/n)*100)

889
38.24521934758155
61.754780652418454


data is not imbalanced

In [70]:
train_data.dtypes

Survived      int64
Pclass        int64
Sex          object
Age         float64
SibSp         int64
Parch         int64
Ticket       object
Fare        float64
Cabin        object
Embarked     object
dtype: object

Now left with 9 features that will determine whether a person survived or not.'Survived'-binary target class

Handling missing values in the dataset in 3 features- Age(177),Cabin(687),Embarked(2)

In [83]:
#drop 'Cabin' feature- too many missing values
train_data=train_data.drop(columns='Cabin')

In [84]:
#for 'Age': impute missing values with -1
train_data['Age'].fillna(-1, inplace=True)

In [95]:
#drop 2 rows where 'Embarked' has missing values
train_data.dropna(subset=['Embarked'], inplace=True)

In [97]:
train_data.isna().sum()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Ticket      0
Fare        0
Embarked    0
dtype: int64

In [102]:
train_data.shape

(889, 9)

# Data Modeling